# Divid the data form the `txt` to the single `csv`.

In [29]:
import csv
import re
import os

input_file = "input.txt"

# Create output folder if not exists
output_folder = "output_csv_files"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

with open(input_file, "r") as f:
    content = f.read()

max_number = 6000  # Set the maximum number to process
pattern = re.compile(r'^(\d+):', re.MULTILINE)
blocks = pattern.split(content)

def process_line(line):
    line_data = []
    if ';' in line:
        sublines = line.split(';')
        for subline in sublines:
            line_data.extend(process_line(subline))
    else:
        if line.startswith('Keywords:'):
            line_data.append(('Keywords', line.split(':', 1)[1].strip()))
        elif '/' in line and '=' in line:
            key_value_pairs = re.findall(r'\/([^=]+)=([^\/]+)', line)
            for key, value in key_value_pairs:
                line_data.append((key.strip(), value.strip() if value.strip() else 'na'))
        elif ':' in line and not ('/' in line and '=' in line):
            key, value = line.rsplit(":", 1)
            line_data.append((key.strip(), value.strip() if value.strip() else 'na'))
    return line_data

file_counter = 1
for i in range(1, len(blocks), 2):
    if file_counter > max_number:
        break

    block = blocks[i+1]
    lines = block.strip().split("\n")
    csv_data = []

    for line in lines:
        if "Accession:" in line and "ID:" in line:
            accession, remaining = line.split("Accession:", 1)[1].strip().split("ID:", 1)
            csv_data.append(('Accession', accession.strip()))
            csv_data.append(('ID', remaining.strip()))
        else:
            csv_data.extend(process_line(line))

    output_file = os.path.join(output_folder, f"{blocks[i]}.csv")
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        # Transpose the csv data and set the first row as column names
        transposed_data = list(zip(*csv_data))
        writer.writerow(transposed_data[0])
        for row in transposed_data[1:]:
            writer.writerow(row)

    file_counter += 1


# Merge the data from the csv

In [37]:
import os
import csv

# 设置输入和输出文件名和路径
input_folder = "output_csv_files"
output_file = "file.csv"

# 初始化合并后csv文件的列名和行数据列表
merged_header = []
merged_rows = []

# 遍历所有csv文件并读取其内容
for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        with open(os.path.join(input_folder, filename), "r") as file:
            # 读取当前文件的第一行并将其添加到合并后的列名列表中
            reader = csv.reader(file)
            header = next(reader)
            if not merged_header:
                merged_header = header
            else:
                for column in header:
                    if column not in merged_header:
                        merged_header.append(column)

            # 读取当前文件的所有行并将其添加到合并后的行数据列表中
            for row in reader:
                merged_row = {}
                for i, value in enumerate(row):
                    merged_row[header[i]] = value
                merged_rows.append(merged_row)

# 将合并后的列名和行数据写入输出文件
with open(output_file, "w") as file:
    writer = csv.DictWriter(file, fieldnames=merged_header)
    writer.writeheader()
    writer.writerows(merged_rows)
# 删除空行
with open(output_file, 'r') as file:
    lines = file.readlines()

with open(output_file, 'w') as file:
    lines = filter(lambda x: x.strip(), lines)
    file.writelines(lines)


        